In [ ]:
path_dict = "/home/jjia1/viralepitope/"
pseq_dict = np.load(path_dict + 'pseq_dict_all.npy', allow_pickle = True).item()

In [ ]:
def convert_alleleseq_to_pseudoseq(seq_dict):
    aa={"A":0,"R":1,"N":2,"D":3,"C":4,"Q":5,"E":6,"G":7,"H":8,"I":9,"L":10,"K":11,"M":12,"F":13,"P":14,"S":15,"T":16,"W":17,"Y":18,"V":19}

    residue_indices = list(range(0,34,1))
    #Now encode the MHC sequences into pseudo-sequences.
    for allele in seq_dict.keys():
        new_pseq = []
        pseq =""
        for index in residue_indices:
            pseq+= seq_dict[allele][index]
            new_pseq.append(seq_dict[allele]) 
        pseq_dict[allele] = new_pseq
    
    return pseq_dict

pseq_dict_converted = convert_alleleseq_to_pseudoseq(pseq_dict)

In [ ]:
def mhc_peptide_pair(path, pseq_dict_matrix):
    aa={"A":0,"R":1,"N":2,"D":3,"C":4,"Q":5,"E":6,"G":7,"H":8,"I":9,"L":10,"K":11,"M":12,"F":13,"P":14,"S":15,"T":16,"W":17,"Y":18,"V":19}
    data_dict = {}
    pep_length = [8,9,10,11,12,13,14,15]
    f = open(path,"r")
    for line in f:
        info = re.split("\t",line)#Retrive information from a tab-delimited line
        allele = info[1].strip()
        if allele in pseq_dict_matrix.keys():
            affinity = int(info[-1].strip()) #Retrive lable information 
            pep = info[0].strip() #Retrive ligand information 
            
            if set(list(pep)).difference(list('ACDEFGHIKLMNPQRSTVWY')):
                print('Illegal peptides')
                continue   
            if len(pep) not in pep_length:
                print('Illegal peptides')
                continue 
                
            new_data = [pep, pseq_dict_matrix[allele], affinity]
            
            if allele not in data_dict.keys():
                data_dict[allele] = [new_data]
            else:
                data_dict[allele].append(new_data)
                
    return data_dict

In [ ]:
pseudo_seq = mhc_peptide_pair('fulldata_withlabels_combined.txt', pseq_dict)

Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides
Illegal peptides


In [ ]:
len(pseudo_seq)

121

In [ ]:
training_data = []
for allele in pseudo_seq.keys():
    allele_data = pseudo_seq[allele]
    random.shuffle(allele_data)
    allele_data = np.array(allele_data)
    training_data.extend(allele_data)

In [ ]:
[all_pep, all_mhc, all_target] = [[i[j] for i in training_data] for j in range(3)]

In [ ]:
all_pepdf = pd.DataFrame(all_pep)
all_mhc_df = pd.DataFrame(all_mhc)
all_target_df = pd.DataFrame(all_target)

In [ ]:
all_pepdf = pd.concat((all_pepdf,all_target_df), axis = 1)

In [ ]:
all_mhcdf = pd.concat((all_mhc_df, all_target_df), axis = 1)

In [ ]:
all_pepdf.columns = ['sequence', 'label']

In [ ]:
all_mhcdf.columns = ['sequence', 'label']

In [ ]:
pos = all_pepdf['label'] == '1'

In [ ]:
all_pepdf_pos = all_pepdf[pos]
all_pepdf_neg = all_pepdf[~pos]

In [ ]:
all_pepdf_pos

,sequence,label
1,TTFDAEYCR,1
31,NYLDYMTSMK,1
53,SISSVLTILY,1
61,MVLAFITFLR,1
64,MTSRMLLNR,1
...,...,...
945269,RRFFPYYVY,1
945275,RRFFPYYV,1
945298,YLEPAIAKY,1
945310,VEFEPPHAA,1


In [ ]:
all_pepdf_neg

,sequence,label
0,PPRVLPLHP,0
2,KKFWVILKG,0
3,LLLHDSYDYD,0
4,EDEEGSESIS,0
5,AGRTKEAGK,0
...,...,...
945319,KKSNSAPSKKA,0
945320,ADKNRARVEEN,0
945321,LVTKTHLLSLS,0
945322,NQQHLQELQQE,0


In [ ]:
all_pepdf_pos.shape

(85934, 2)

In [ ]:
all_pepdf_pos = all_pepdf_pos.reset_index()
all_pepdf_neg = all_pepdf_neg.reset_index()

In [ ]:
ofile = open('positive_viralepitope_sequenceonly.txt', 'w')
for i in range(all_pepdf_pos.shape[0]):
    ofile.write('>Sequence_'+str(i)+'\n'+all_pepdf_pos['sequence'][i]+'\n')
ofile.close()

In [ ]:
ofile = open('negative_viralepitope_sequenceonly.txt', 'w')
for i in range(all_pepdf_neg.shape[0]):
    ofile.write('>Sequence_'+str(i)+'\n'+all_pepdf_neg['sequence'][i]+'\n')
ofile.close()

In [ ]:
#generating sgt embedding

In [ ]:
pep_df = pd.DataFrame(all_pep)
mhc_df = pd.DataFrame(all_mhc)
target_df = pd.DataFrame(all_target)

In [ ]:
pep_df.columns = ['sequence']
pep_df['sequence'] = pep_df['sequence'].map(list)
pep_df

,sequence
0,"[P, P, R, V, L, P, L, H, P]"
1,"[T, T, F, D, A, E, Y, C, R]"
2,"[K, K, F, W, V, I, L, K, G]"
3,"[L, L, L, H, D, S, Y, D, Y, D]"
4,"[E, D, E, E, G, S, E, S, I, S]"
...,...
945319,"[K, K, S, N, S, A, P, S, K, K, A]"
945320,"[A, D, K, N, R, A, R, V, E, E, N]"
945321,"[L, V, T, K, T, H, L, L, S, L, S]"
945322,"[N, Q, Q, H, L, Q, E, L, Q, Q, E]"


In [ ]:
mhc_df.columns = ['sequence']
mhc_df['sequence'] = mhc_df['sequence'].map(list)
mhc_df

,sequence
0,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
1,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
2,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
3,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
4,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
...,...
945319,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."
945320,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."
945321,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."
945322,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."


In [ ]:
pep_df['id'] = pep_df.index
mhc_df['id'] = mhc_df.index

In [ ]:
pep_df = pep_df[pep_df.columns[::-1]]
pep_df

,id,sequence
0,0,"[P, P, R, V, L, P, L, H, P]"
1,1,"[T, T, F, D, A, E, Y, C, R]"
2,2,"[K, K, F, W, V, I, L, K, G]"
3,3,"[L, L, L, H, D, S, Y, D, Y, D]"
4,4,"[E, D, E, E, G, S, E, S, I, S]"
...,...,...
945319,945319,"[K, K, S, N, S, A, P, S, K, K, A]"
945320,945320,"[A, D, K, N, R, A, R, V, E, E, N]"
945321,945321,"[L, V, T, K, T, H, L, L, S, L, S]"
945322,945322,"[N, Q, Q, H, L, Q, E, L, Q, Q, E]"


In [ ]:
mhc_df = mhc_df[mhc_df.columns[::-1]]
mhc_df

,id,sequence
0,0,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
1,1,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
2,2,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
3,3,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
4,4,"[Y, T, A, M, Y, R, N, N, V, A, H, I, D, V, D, ..."
...,...,...
945319,945319,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."
945320,945320,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."
945321,945321,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."
945322,945322,"[Y, H, T, K, Y, R, E, I, S, T, N, T, Y, E, S, ..."


In [ ]:
target_df['id'] = target_df.index
target_df = target_df[target_df.columns[::-1]]

In [ ]:
target_df = target_df[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder


encoder = LabelEncoder()
encoder.fit(all_target)
encoded_target = encoder.transform(all_target)

In [ ]:
encoded_target

array([0, 1, 0, ..., 0, 0, 0])

In [ ]:
import pandarallel
from sgt import SGT

In [ ]:
alphabet = list('ACDEFGHIKLMNPQRSTVWY')

In [ ]:
sgt_ = SGT(alphabets = alphabet, kappa =5, lengthsensitive = True, mode = 'multiprocessing', processors= 8)

In [ ]:
pep_sgtembedding = sgt_.fit_transform(pep_df)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
pep_sgtembedding

,id,"(A, A)","(A, C)","(A, D)","(A, E)","(A, F)","(A, G)","(A, H)","(A, I)","(A, K)",...,"(Y, M)","(Y, N)","(Y, P)","(Y, Q)","(Y, R)","(Y, S)","(Y, T)","(Y, V)","(Y, W)","(Y, Y)"
0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
1,1.0,0.000000,0.077262,0.00000,0.570892,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.210019,0.0,0.0,0.0,0.0,0.000000
2,2.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,3.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.214492
4,4.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945319,945319.0,0.010884,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.070109,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
945320,945320.0,0.010884,0.000000,0.59427,0.061033,0.0,0.0,0.0,0.0,0.218620,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
945321,945321.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
945322,945322.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [ ]:
mhc_sgtembedding = sgt_.fit_transform(mhc_df)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
mhc_sgtembedding

,id,"(A, A)","(A, C)","(A, D)","(A, E)","(A, F)","(A, G)","(A, H)","(A, I)","(A, K)",...,"(Y, M)","(Y, N)","(Y, P)","(Y, Q)","(Y, R)","(Y, S)","(Y, T)","(Y, V)","(Y, W)","(Y, Y)"
0,0.0,0.070434,0.0,0.070435,0.0,0.0,0.0,0.52044,0.207632,0.0,...,0.207911,0.207911,0.0,0.564403,0.648329,0.000000,0.547268,0.027446,0.194820,0.067405
1,1.0,0.070434,0.0,0.070435,0.0,0.0,0.0,0.52044,0.207632,0.0,...,0.207911,0.207911,0.0,0.564403,0.648329,0.000000,0.547268,0.027446,0.194820,0.067405
2,2.0,0.070434,0.0,0.070435,0.0,0.0,0.0,0.52044,0.207632,0.0,...,0.207911,0.207911,0.0,0.564403,0.648329,0.000000,0.547268,0.027446,0.194820,0.067405
3,3.0,0.070434,0.0,0.070435,0.0,0.0,0.0,0.52044,0.207632,0.0,...,0.207911,0.207911,0.0,0.564403,0.648329,0.000000,0.547268,0.027446,0.194820,0.067405
4,4.0,0.070434,0.0,0.070435,0.0,0.0,0.0,0.52044,0.207632,0.0,...,0.207911,0.207911,0.0,0.564403,0.648329,0.000000,0.547268,0.027446,0.194820,0.067405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945319,945319.0,0.100789,0.0,0.273972,0.0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.073050,0.0,0.000000,0.648329,0.202799,0.470527,0.025158,0.183226,0.364197
945320,945320.0,0.100789,0.0,0.273972,0.0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.073050,0.0,0.000000,0.648329,0.202799,0.470527,0.025158,0.183226,0.364197
945321,945321.0,0.100789,0.0,0.273972,0.0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.073050,0.0,0.000000,0.648329,0.202799,0.470527,0.025158,0.183226,0.364197
945322,945322.0,0.100789,0.0,0.273972,0.0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.073050,0.0,0.000000,0.648329,0.202799,0.470527,0.025158,0.183226,0.364197


In [ ]:
import pickle as pkl
pkl.dump(pep_sgtembedding, open('peptide-sgt-embedding.pkl', 'wb'))
pkl.dump(mhc_sgtembedding, open('mhc-sgt-embedding.pkl', 'wb'))